In [6]:
#!/usr/bin/env python

from astropy.io import fits
from astropy.table import Table


import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import scipy.integrate
import astropy.units as u
import GCRCatalogs
import pandas as pd
from astropy.io import fits
from astropy.table import Table

from numcosmo_py import Ncm, Nc
from numcosmo_py.external.pyssc import pyssc as PySSC
Ncm.cfg_init()


%matplotlib inline
catalog = "skysim5000_v1.1.1"
skysim_cat = GCRCatalogs.load_catalog(catalog)
cosmo_ss = skysim_cat.cosmology

In [11]:
cosmo = Nc.HICosmoDEXcdm()
reion = Nc.HIReionCamb.new()
prim = Nc.HIPrimPowerLaw.new()

cosmo.add_submodel(reion)
cosmo.add_submodel(prim)

dist = Nc.Distance.new(2.0)

tf = Nc.TransferFuncEH()

psml = Nc.PowspecMLTransfer.new(tf)
psml.require_kmin(1.0e-6)
psml.require_kmax(1.0e3)

psf = Ncm.PowspecFilter.new(psml, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0()


cosmo.props.H0 = cosmo_ss.H0.value
cosmo.props.Omegab = cosmo_ss.Ob0
cosmo.props.Omegac = cosmo_ss.Odm0
cosmo.props.Omegax = cosmo_ss.Ode0

cosmo.omega_x2omega_k()
cosmo.param_set_by_name("Omegak", 0.0)

prim.props.n_SA = cosmo_ss.n_s
print(cosmo_ss.sigma8, cosmo.sigma8(psf), cosmo.Omega_k0())

old_amplitude = np.exp(prim.props.ln10e10ASA)
prim.props.ln10e10ASA = np.log(
    (cosmo_ss.sigma8 / cosmo.sigma8(psf)) ** 2 * old_amplitude
)
print(cosmo_ss.sigma8, cosmo.sigma8(psf))

0.8 0.8490796381179414 0.0
0.8 0.7999999999999999


In [16]:
# CosmoSim_proxy model
# M_0, z_0
#theta_pivot = [3e14 / 0.71, 0.6]
# \mu_0, a_\mu^z, a_\mu^M
#theta_mu = [3.19, -0.7, 2]
# \sigma_0, a_\sigma^z, a_\sigma^M
#theta_sigma = [0.33, 0.0, -0.08]

area = 439.78987
lnM_min = np.log(10) * (13)
lnM_max =  np.log(10.0) * (15.0)
z_min = 0.0
z_max = 1.0

#mass proxy
cluster_m = Nc.ClusterMassNodist(lnM_min=lnM_min, lnM_max=lnM_max)

#photo-z
cluster_z = Nc.ClusterRedshiftNodist (z_min = z_min, z_max = z_max)

In [18]:
# Numcosmo Cluster Abundance

# First we need to define the multiplicity function here we will use the tinker
mulf = Nc.MultiplicityFuncDespali.new()
mulf.set_mdef(Nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta(200)
# Second we need to construct a filtered power spectrum

hmf = Nc.HaloMassFunction.new(dist, psf, mulf)
hmf.set_area_sd(area)


ca = Nc.ClusterAbundance.new(hmf, None)
mset = Ncm.MSet.new_array([cosmo, cluster_m, cluster_z])

In [ ]:
ncount = Nc.DataClusterNCount.new(ca, "NcClusterRedshiftNodist", "NcClusterMassNodist")

ca.prepare(cosmo, cluster_z, cluster_m)
mset = Ncm.MSet.new_array([cosmo, cluster_z, cluster_m])

rng = Ncm.RNG.seeded_new(None, 42)

ncount.init_from_sampling(mset, area * ((np.pi / 180) ** 2), rng)
print(ncount.desc)

In [ ]:
# Bins definition
z_edges = np.linspace(zl, zu, 5)
# proxy_edges = np.linspace(lnRl, lnRu, 4)
mass_edges = np.linspace(math.log(10) * 13, math.log(10) * 15, 4)

ncount.del_bins()
for rich_l, rich_u in zip(mass_edges, mass_edges[1:]):
    for zl, zu in zip(z_edges, z_edges[1:]):
        lnM_obs_lb = Ncm.Vector.new_array([rich_l])
        lnM_obs_ub = Ncm.Vector.new_array([rich_u])
        z_obs_lb = Ncm.Vector.new_array([zl])
        z_obs_ub = Ncm.Vector.new_array([zu])

        ncount.add_bin(lnM_obs_lb, lnM_obs_ub, z_obs_lb, z_obs_ub)

ncount.bin_data()

In [ ]:
ncount.catalog_save("ncount_ascaso2.fits", True)
ncount.catalog_load("ncount_ascaso2.fits")

# ncount.catalog_save ("ncount_nodist.fits", True)
# ncount.catalog_load ("ncount_nodist.fits")

In [ ]:
ncdata_fits = fits.open("ncount_ascaso2.fits")
# ncdata_fits.info()

ncdata_data = ncdata_fits[1].data

ncdata_Table = Table(ncdata_data)
ncdata_Table.columns
# ncdata_Table.sort('LNM_TRUE')

display(ncdata_Table)

In [12]:
cluster_m = Nc.ClusterMass.new_from_name(
    "NcClusterMassNodist{'lnM-min':<%20.15e>, 'lnM-max':<%20.15e>}"
    % (math.log(10) * np.log10(1e13), math.log(10) * np.log10(1e15))
)
ncount_nodist = Nc.DataClusterNCount.new(
    ca, "NcClusterRedshiftNodist", "NcClusterMassNodist"
)
ca.prepare(cosmo, cluster_z, cluster_m)
mset = Ncm.MSet.new_array([cosmo, cluster_z, cluster_m])

hmf.set_area_sd(skysim_cat.sky_area)
ncount_nodist.set_lnM_true(Ncm.Vector.new_array(ncdata_Table["LNM_TRUE"]))
ncount_nodist.set_z_true(Ncm.Vector.new_array(ncdata_Table["Z_TRUE"]))

# Because we are using true masses and redshifts in this example,
# we replicate the true data in the 'observed' masses and redshift attributes
ncount_nodist.set_lnM_obs(Ncm.Matrix.new_array(ncdata_Table["LNM_TRUE"], 1))
ncount_nodist.set_z_obs(Ncm.Matrix.new_array(ncdata_Table["Z_TRUE"], 1))
ncount_nodist.props.area = skysim_cat.sky_area
ncount_nodist.true_data(True)
ncount_nodist.set_init(True)

print(ncount_nodist.desc)

ncount_nodist.catalog_save("ncount_nodist.fits", True)
ncount_nodist.catalog_load("ncount_nodist.fits")

ncdata_fits = fits.open("ncount_nodist.fits")
print(ncount_nodist.desc)


# ncdata_fits.info()

ncdata_data = ncdata_fits[1].data

ncdata_Table = Table(ncdata_data)
ncdata_Table.columns
ncdata_Table.sort("LNM_TRUE")

display(ncdata_Table)

AttributeError: type object 'ClusterMass' has no attribute 'new_from_name'